In [7]:
import re
import sys
from pathlib import Path

import os.path
import PyPDF2
import pdfminer
import pdfplumber
import PySimpleGUI as sg

import constants

import logging

In [8]:
logger = logging.getLogger('pypdf_splitter')
logger.addHandler(logging.StreamHandler(sys.stdout))
logger.setLevel(logging.DEBUG)

In [14]:
!jupyter-nbconvert --to python --template python_clean pypdf_split.ipynb

[NbConvertApp] Converting notebook pypdf_split.ipynb to python
[NbConvertApp] Writing 4088 bytes to pypdf_split.py


In [10]:
class PyPDFSplitter:
    # contributed by picobas
    def get_breaks(file):
        file = Path(file).expanduser().absolute()

        try:
            source_pdf = pdfplumber.open(file);
        except (FileNotFoundError, pdfminer.pdfparser.PDFSyntaxError, IsADirectoryError) as e:
            logger.error(f'Failed to read source pdf: {e}')
            return []

        page_breaks = []

        last_record = {'id': None, 'pages': []}

        for page in range(0, len(source_pdf.pages)):
            this_record = ({'id': None, 'pages': []})
            text = source_pdf.pages[page].extract_text()
            student_number = re.search('student\W?id:\W?(\d{6})', text, re.IGNORECASE)

            if student_number:
                logger.debug(f"Found student: {student_number.group(1)}")
                page_breaks.append(last_record)
                this_record['id'] = student_number.group(1)
                this_record['pages'].append(page)
                last_record = this_record

            else:
                last_record['pages'].append(page)

        return page_breaks

    def split_pdf(file, page_breaks):
        file = Path(file).expanduser().absolute()
        file_name = os.path.splitext(file.name)[0]
        dest_path = file.parent / f'{file_name}_split'

        output = {}

        try:
            source_pdf = PyPDF2.PdfFileReader(str(file))
        except (FileNotFoundError, PyPDF2.utils.PdfReadError, IsADirectoryError) as e:
            logger.error(f'Failed to split source pdf: {e}')
            return False

        for record in page_breaks:
            output_path = 'Failed'
            if not record['id']:
                continue
            pdf_writer = PyPDF2.PdfFileWriter()
            for page in record['pages']:
                try:
                    pdf_writer.addPage(source_pdf.getPage(page))
                except Exception as e:
                    logger.error(f'failed to add page: "{page}" for record: "{record}" error: {e}')

            try:
                output_path = dest_path / f'{record["id"]} - {file_name}.pdf'
                output_path.parent.mkdir(exist_ok=True)
                with open(str(output_path), "wb") as dest_file:
                    pdf_writer.write(dest_file)

            except Exception as e:
                logger.error(f'error outputing pdf: {e}')
                output_path = f'Failed outputing pdf: {e}'
                pass
            finally:
                output[record['id']] = output_path
        return output

In [11]:
def main():
    interactive = False

    if (len(sys.argv) == 1) or ('-f' in sys.argv):
        interactive = True
    else:
        fname = sys.argv[1]

    while interactive:
        logger.debug('prompt user for PDF file')
        fname = sg.popup_get_file(f'v{constants.version}: Choose a PDF to split')
        logger.debug(f'fname returned: {fname}')
        if fname in [None, ]:
            logger.info('user canceled')
            break
        if fname:
            breaks = PyPDFSplitter.get_breaks(fname)
            output = PyPDFSplitter.split_pdf(fname, breaks)

        if output:
            ## add a popup that summarizes what was done
            strings = f'{len(output)} records extracted -- see below for full output\n'
            strings = strings + 'split records are placed in the same directory as the original PDF\n\n\n'
            strings = strings + str(output)
            sg.popup_scrolled(strings,
                              title='Processed files',
                              keep_on_top=True,
                              size=(80, 50))
        else:
            sg.popup('Cancel', "No PDF supplied")

In [12]:
if __name__ == "__main__":
    main()

prompt user for PDF file
fname returned: /Users/aaronciuffo/Downloads/drive-download-20210706T083053Z-001/Gr5_Sem2_Report Card.pdf
Found student: 503252
Found student: 503624
Found student: 503842
Found student: 504372
Found student: 504569
Found student: 504642
Found student: 504699
Found student: 505466
Found student: 505506
Found student: 505548
Found student: 505592
Found student: 505867
Found student: 503268
Found student: 503522
Found student: 504374
Found student: 504567
Found student: 504868
Found student: 504871
Found student: 504987
Found student: 504989
Found student: 505203
Found student: 505344
Found student: 505359
Found student: 505463
Found student: 505525
Found student: 505574
Found student: 505681
Found student: 503614
Found student: 503626
Found student: 504377
Found student: 504680
Found student: 504985
Found student: 505775
Found student: 504570
Found student: 505862
Found student: 505876
Found student: 504363
Found student: 504373
Found student: 504678
Found stude